# **NER_by_BERT**

## **Author:** Gema De Vargas Romero

## **Master Thesis:** "Development of a Named Entity Recognition System to automatically assign tumor morphology entity mentions to health-related documents in Spanish." 

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

path='drive/My Drive/Ejemplos NER - TFM/'
!ls 'drive/My Drive/Ejemplos NER - TFM/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
 bert
 data
 dev_set
 dev_set2
'Dictionary based NER (spacy).ipynb'
'Ehealth_Dictionary based NER (spacy).ipynb'
 last_step_cantemist.ipynb
 last_step_cantemist_TEST.ipynb
 NER_by_BERT_Cantemist_BIOESV.ipynb
 NER_by_BERT_Cantemist_Competicion.ipynb
 NER_by_BERT_Cantemist.ipynb
 NER_by_BI_LSTM_CRF_Cantemist_BIOESV_2.ipynb
 NER_by_BI_LSTM_CRF_Cantemist_BIOESV.ipynb
 NER_by_BI_LSTM_CRF_Cantemist_Competicion.ipynb
 NER_by_BI_LSTM_CRF_Cantemist.ipynb
 NER_by_CRF_Cantemist_Competicion.ipynb
 NER_by_CRF_Cantemist.ipynb
 NER_by_CRF_Ehealth.ipynb
 NER_by_CRF.ipynb
 Preprocessing_NER_Cantemist.ipynb
 resources
 results_bert
 results_bert2
 results_BILSTM_ap1
 results_BILSTM_ap2
 results_BILSTM_ap3
 results_CRF
 sample_set
 Scielo+Wiki_skipgram_cased.bin
 Scielo+Wiki_skipgram_cased.vec
 test-background-set-to-publish
 train_set


## **Loading libraries**

First, we must load the libraries that we will use in this notebook:

In [ ]:
!pip install sklearn-crfsuite
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_precision_score
from sklearn_crfsuite.metrics import flat_recall_score
from sklearn_crfsuite.metrics import flat_classification_report
import numpy as np
import pandas as pd
import time
import random
#import nltk
import matplotlib.pyplot as plt

     |████████████████████████████████| 747kB 3.9MB/s 


In [ ]:
# Library spacy
!pip install -U spacy 
#!python -m spacy validate
!python -m spacy download es_core_news_lg

import spacy

# nlp = spacy.load("es") # no longer works with updated version of spacy 2.3.1
import es_core_news_lg
nlp = es_core_news_lg.load()

     |████████████████████████████████| 10.0MB 2.8MB/s 
     |████████████████████████████████| 2.1MB 38.4MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 573.1MB 2.1MB/s 
  Created wheel for es-core-news-lg: filename=es_core_news_lg-2.3.1-cp36-none-any.whl size=573139081 sha256=ede5718f5d0f5c53ccbcacdc5f445898c6e041a521919eb01c0e2638782fec8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-9e6puq8v/wheels/48/59/33/558e7f48e924c6cac0cbd3679ee7c84f5ae02964c335232e5a
Successfully built es-core-news-lg
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_lg')


## **Read the files**

In [ ]:
!ls 'drive/My Drive/Ejemplos NER - TFM/data'
import pickle as pkl

df_data_complete.csv  files_txt_dev	   sentences_dev_by_cc2
df_data_dev2_2.csv    files_txt_test	   sentences_test
df_data_dev2.csv      sentences_dev	   sentences_test_by_cc
df_data_test.csv      sentences_dev2	   sentences_train
df_data_train2.csv    sentences_dev_by_cc


In [ ]:
# Train dataset
with open(path+'data/sentences_train', 'rb') as file: 
  sentences_train = pkl.load(file)
file.close()

df_data_train2 = pd.read_csv(path+'data/df_data_train2.csv')

In [ ]:
# Development dataset 1
with open(path+'data/sentences_dev', 'rb') as file: 
  sentences_dev = pkl.load(file)
file.close()

with open(path+'data/sentences_dev_by_cc', 'rb') as file: 
  sentences_dev_by_cc = pkl.load(file)
file.close()

df_data_dev2 = pd.read_csv(path+'data/df_data_dev2.csv')

In [ ]:
# Development dataset 2
with open(path+'data/sentences_dev2', 'rb') as file: 
  sentences_dev2 = pkl.load(file)
file.close()

with open(path+'data/sentences_dev_by_cc2', 'rb') as file: 
  sentences_dev_by_cc2 = pkl.load(file)
file.close()

df_data_dev2_2 = pd.read_csv(path+'data/df_data_dev2_2.csv')

In [ ]:
# TEST FILES:
with open(path+'data/sentences_test', 'rb') as file: 
  sentences_test = pkl.load(file)
file.close()

with open(path+'data/sentences_test_by_cc', 'rb') as file: 
  sentences_test_by_cc = pkl.load(file)
file.close()

df_data_test = pd.read_csv(path+'data/df_data_test.csv')

In [ ]:
df_data_train2 = df_data_train2.drop_duplicates(['File_Index','Sentence_Index','Word','start'], keep = 'first')
df_data_dev2 = df_data_dev2.drop_duplicates(['File_Index','Sentence_Index','Word','start'], keep = 'first')
df_data_dev2_2 = df_data_dev2_2.drop_duplicates(['File_Index','Sentence_Index','Word','start'], keep = 'first')

## **Construct Complete dataset. Combine train and development sets 1 and 2**

This will construct the full dataframe that will be employed to train the final model.

In [ ]:
df_data_train2

,File_Index,Sentence_Index,Word,POS,start,Tag
0,1,1,ANAMNESIS,NOUN,0,O
1,1,1,\n,SPACE,9,O
2,1,1,Mujer,NOUN,10,O
3,1,1,de,ADP,16,O
4,1,1,67,NUM,19,O
...,...,...,...,...,...,...
454525,501,19502,deshidrogenasa,PROPN,7856,O
454526,501,19502,(,PUNCT,7871,O
454527,501,19502,SDHB,PROPN,7872,O
454528,501,19502,),PUNCT,7876,O


In [ ]:
file_idx = df_data_train2['File_Index'].iloc[-1]
sent_idx = df_data_train2['Sentence_Index'].iloc[-1]
file_idx2 = df_data_dev2['File_Index'].iloc[-1]
sent_idx2 = df_data_dev2['Sentence_Index'].iloc[-1]

print(file_idx)
print(sent_idx)
print(file_idx2)
print(sent_idx2)

501
19502
250
9546


In [ ]:
df_data_dev2_aux = df_data_dev2.copy()
df_data_dev2_2_aux = df_data_dev2_2.copy()

In [ ]:
df_data_dev2_aux['File_Index'] = df_data_dev2['File_Index'] + file_idx
df_data_dev2_aux['Sentence_Index'] = df_data_dev2['Sentence_Index'] + sent_idx

df_data_dev2_2_aux['File_Index'] = df_data_dev2_2['File_Index'] + file_idx + file_idx2
df_data_dev2_2_aux['Sentence_Index'] = df_data_dev2_2['Sentence_Index'] + sent_idx + sent_idx2

In [ ]:
df_data_dev2_2_aux

,File_Index,Sentence_Index,Word,POS,start,Tag
0,752,29049,Anamnesis,NOUN,0,O
1,752,29049,\n,SPACE,9,O
2,752,29049,Mujer,NOUN,10,O
3,752,29049,de,ADP,16,O
4,752,29049,60,NUM,19,O
...,...,...,...,...,...,...
183180,1001,37730,exitus,NOUN,5646,O
183181,1001,37730,de,ADP,5653,O
183182,1001,37730,la,DET,5656,O
183183,1001,37730,paciente,NOUN,5659,O


In [ ]:
df_data_complete = df_data_train2.append(df_data_dev2_aux, ignore_index=True)
df_data_complete = df_data_complete.append(df_data_dev2_2_aux, ignore_index=True)

In [ ]:
df_data_complete

,File_Index,Sentence_Index,Word,POS,start,Tag
0,1,1,ANAMNESIS,NOUN,0,O
1,1,1,\n,SPACE,9,O
2,1,1,Mujer,NOUN,10,O
3,1,1,de,ADP,16,O
4,1,1,67,NUM,19,O
...,...,...,...,...,...,...
862674,1001,37730,exitus,NOUN,5646,O
862675,1001,37730,de,ADP,5653,O
862676,1001,37730,la,DET,5656,O
862677,1001,37730,paciente,NOUN,5659,O


In [ ]:
df_data_complete.to_csv(path+'data/df_data_complete.csv', index = False, header=True)

## **Model 3: BERT**

https://medium.com/dair-ai/beto-spanish-bert-420e4860d2c6

https://github.com/dccuchile/beto

#### **Loading libraries**

In [ ]:
!pip install transformers==2.6.0

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

     |████████████████████████████████| 542kB 2.8MB/s 
     |████████████████████████████████| 890kB 12.9MB/s 
     |████████████████████████████████| 1.1MB 17.4MB/s 
     |████████████████████████████████| 3.7MB 25.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a556b6d0b903372693432d30bc55033fa3d13d577507dab574fdd17c633d66bc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Using TensorFlow backend.


'2.6.0'

#### **Model parameters**

Recommended parameters by the paper:

- batch size: 16, 32
- Learning Rate (Adam): 5e-5, 3e-5, 2e-5
- Number of Epochs: 2, 3, 4

Perform hyper-parameter tuning to find the best combination of these parameters. FUTURE WORK --> google cloud computing

In [ ]:
max_len = 75 # sequence length limited to 75 tokens 
batch_size = 32 # suggested 32
# Bert supports sequences of up to 512 tokens.

In [ ]:
# to work with GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

#### **Preprocessing the data**

In [ ]:
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t, i) for w, p, t, i in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist(),
                                                       s['start'].values.tolist())]
        self.grouped = self.df.groupby("Sentence_Index").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [ ]:
# This is a class to get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence_test(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, i) for w, p, i in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                      s['Start_Char_position'].values.tolist())]
        self.grouped = self.df.groupby("Sentence_Index").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

**Prepare the sentences and labels**

Before we can start fine-tuning the model, we have to prepare the data set for the use with pytorch and BERT.

The sentences employed for BERT **sentences_bert** do not contain the POS and TAG values present in previous models because they will be tokenized by BERT. In fact, the corresponding TAGs of each sentence will be stores in **labels**. 

In [ ]:
df_data_train2

,File_Index,Sentence_Index,Word,POS,start,Tag
0,1,1,ANAMNESIS,NOUN,0,O
1,1,1,\n,SPACE,9,O
2,1,1,Mujer,NOUN,10,O
3,1,1,de,ADP,16,O
4,1,1,67,NUM,19,O
...,...,...,...,...,...,...
454525,501,19502,deshidrogenasa,PROPN,7856,O
454526,501,19502,(,PUNCT,7871,O
454527,501,19502,SDHB,PROPN,7872,O
454528,501,19502,),PUNCT,7876,O


##### **Complete dataset**

combines train and development datasets

In [ ]:
df_data_complete

,File_Index,Sentence_Index,Word,POS,start,Tag
0,1,1,ANAMNESIS,NOUN,0,O
1,1,1,\n,SPACE,9,O
2,1,1,Mujer,NOUN,10,O
3,1,1,de,ADP,16,O
4,1,1,67,NUM,19,O
...,...,...,...,...,...,...
862674,1001,37730,exitus,NOUN,5646,O
862675,1001,37730,de,ADP,5653,O
862676,1001,37730,la,DET,5656,O
862677,1001,37730,paciente,NOUN,5659,O


In [ ]:
# FULL DATASET (train and development sets combined)

# sentences
getter = sentence(df_data_complete)
sentences_bert_complete = [[s[0] for s in sentence] for sentence in getter.sentences]
print(sentences_bert_complete[0])

# labels
labels_bert_complete = [[s[2] for s in sentence] for sentence in getter.sentences]
print(labels_bert_complete[0])

# start char position
start_bert_complete = [[s[3] for s in sentence] for sentence in getter.sentences]
print(start_bert_complete[0])

# Tags
tag_values = list(set(df_data_complete["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
print(tag_values)
print(tag2idx)

['ANAMNESIS', '\n', 'Mujer', 'de', '67', 'años', 'con', 'antecedentes', 'personales', 'de', 'hipotiroidismo', 'en', 'tratamiento', 'con', 'levotiroxina', 'y', 'fumadora', 'activa', 'de', '12.5', 'paquetes', '/', 'año', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[0, 9, 10, 16, 19, 22, 27, 31, 44, 55, 58, 73, 76, 88, 92, 105, 107, 116, 123, 126, 131, 139, 140, 143]
['O', 'S-MOR', 'B-MOR', 'E-MOR', 'V-MOR', 'I-MOR', 'PAD']
{'O': 0, 'S-MOR': 1, 'B-MOR': 2, 'E-MOR': 3, 'V-MOR': 4, 'I-MOR': 5, 'PAD': 6}


##### **Train and development sets**

In [ ]:
# TRAIN SET

# sentences
getter = sentence(df_data_train2)
sentences_bert_train = [[s[0] for s in sentence] for sentence in getter.sentences]
print(sentences_bert_train[0])

# labels
labels_bert_train = [[s[2] for s in sentence] for sentence in getter.sentences]
print(labels_bert_train[0])

# start char position
start_bert_train = [[s[3] for s in sentence] for sentence in getter.sentences]
print(start_bert_train[0])

# Tags
tag_values = list(set(df_data_train2["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
print(tag_values)
print(tag2idx)

['ANAMNESIS', '\n', 'Mujer', 'de', '67', 'años', 'con', 'antecedentes', 'personales', 'de', 'hipotiroidismo', 'en', 'tratamiento', 'con', 'levotiroxina', 'y', 'fumadora', 'activa', 'de', '12.5', 'paquetes', '/', 'año', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[0, 9, 10, 16, 19, 22, 27, 31, 44, 55, 58, 73, 76, 88, 92, 105, 107, 116, 123, 126, 131, 139, 140, 143]
['O', 'V-MOR', 'B-MOR', 'E-MOR', 'S-MOR', 'I-MOR', 'PAD']
{'O': 0, 'V-MOR': 1, 'B-MOR': 2, 'E-MOR': 3, 'S-MOR': 4, 'I-MOR': 5, 'PAD': 6}


In [ ]:
# DEVELOPMENT SET

# *** without considering clinical cases independently ***

# sentences
getter_dev = sentence(df_data_dev2)
sentences_bert_dev = [[s[0] for s in sentence] for sentence in getter_dev.sentences]
print(sentences_bert_dev[0])

# labels
labels_bert_dev = [[s[2] for s in sentence] for sentence in getter_dev.sentences]
print(labels_bert_dev[0])

# start char position
start_bert_dev = [[s[3] for s in sentence] for sentence in getter_dev.sentences]
print(start_bert_dev[0])

# ----------------

# *** By clinical case ***
getter_by_cc = df_data_dev2.groupby("File_Index").apply(sentence)

sentences_bert_dev_by_cc = []
labels_bert_dev_by_cc = []
start_bert_dev_by_cc = []

for getter_i in getter_by_cc: # iterating over all the files
  # sentences
  sent_by_cc = [[s[0] for s in sentence] for sentence in getter_i.sentences]
  sentences_bert_dev_by_cc.append(sent_by_cc)
  # labels
  lab_by_cc = [[s[2] for s in sentence] for sentence in getter_i.sentences]
  labels_bert_dev_by_cc.append(lab_by_cc)
  # start char position
  start_by_cc = [[s[3] for s in sentence] for sentence in getter_i.sentences]
  start_bert_dev_by_cc.append(start_by_cc)

print()
print(sentences_bert_dev_by_cc[0][0])
print(labels_bert_dev_by_cc[0][0])
print(start_bert_dev_by_cc[0][0])

['Anamnesis', '\n', 'Varón', 'de', '74', 'años', ',', 'exfumador', 'desde', 'hace', '15', 'años', ',', 'con', 'único', 'antecedente', 'de', 'hipertensión', ',', 'dislipemia', 'y', 'apendicectomizado', ';', 'se', 'diagnostica', 'en', 'marzo', 'de', '2013', 'de', 'carcinoma', 'de', 'células', 'transicionales', 'de', 'vejiga', 'E-IV', '(', 'pulmonares', 'y', 'óseas', ')', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'E-MOR', 'O', 'O', 'O', 'O', 'O']
[0, 9, 10, 16, 19, 22, 26, 28, 38, 44, 49, 52, 56, 58, 62, 68, 80, 83, 95, 97, 108, 110, 127, 129, 132, 144, 147, 153, 156, 161, 164, 174, 177, 185, 200, 203, 210, 215, 216, 227, 229, 234, 235]

['Anamnesis', '\n', 'Varón', 'de', '74', 'años', ',', 'exfumador', 'desde', 'hace', '15', 'años', ',', 'con', 'único', 'antecedente', 'de', 'hipertensión', ',', 'dislipemia', 'y', 

In [ ]:
# DEVELOPMENT SET 2

# *** without considering clinical cases independently ***

# sentences
getter_dev2 = sentence(df_data_dev2_2)
sentences_bert_dev2 = [[s[0] for s in sentence] for sentence in getter_dev2.sentences]
print(sentences_bert_dev2[0])

# labels
labels_bert_dev2 = [[s[2] for s in sentence] for sentence in getter_dev2.sentences]
print(labels_bert_dev2[0])

# start char position
start_bert_dev2 = [[s[3] for s in sentence] for sentence in getter_dev2.sentences]
print(start_bert_dev2[0])
# ----------------

# *** By clinical case ***
getter_by_cc2 = df_data_dev2_2.groupby("File_Index").apply(sentence)

sentences_bert_dev_by_cc2 = []
labels_bert_dev_by_cc2 = []
start_bert_dev_by_cc2 = []

for getter_i in getter_by_cc2: # iterating over all the files
  # sentences
  sent_by_cc = [[s[0] for s in sentence] for sentence in getter_i.sentences]
  sentences_bert_dev_by_cc2.append(sent_by_cc)
  # labels
  lab_by_cc = [[s[2] for s in sentence] for sentence in getter_i.sentences]
  labels_bert_dev_by_cc2.append(lab_by_cc)
  # start char position
  start_by_cc = [[s[3] for s in sentence] for sentence in getter_i.sentences]
  start_bert_dev_by_cc2.append(start_by_cc)

print()
print(sentences_bert_dev_by_cc2[0][0])
print(labels_bert_dev_by_cc2[0][0])

['Anamnesis', '\n', 'Mujer', 'de', '60', 'años', 'diagnosticada', 'de', 'enfermedad', 'pulmonar', 'obstructiva', 'crónica', 'y', 'trastorno', 'bipolar', 'de', 'años', 'de', 'evolución', ',', 'en', 'tratamiento', 'con', 'carbamazepina', 'hasta', 'la', 'fecha', ',', 'sin', 'otros', 'antecedentes', 'personales', 'de', 'interés', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[0, 9, 10, 16, 19, 22, 27, 41, 44, 55, 64, 76, 84, 86, 96, 104, 107, 112, 115, 124, 126, 129, 141, 145, 159, 165, 168, 173, 175, 179, 185, 198, 209, 212, 219]

['Anamnesis', '\n', 'Mujer', 'de', '60', 'años', 'diagnosticada', 'de', 'enfermedad', 'pulmonar', 'obstructiva', 'crónica', 'y', 'trastorno', 'bipolar', 'de', 'años', 'de', 'evolución', ',', 'en', 'tratamiento', 'con', 'carbamazepina', 'hasta', 'la', 'fecha', ',', 'sin', 'otros', 'antecedentes', 'personales', 'de', 'interés', '.'

In [ ]:
print(sentences_bert_dev_by_cc2[0][1])
print(sentences_bert_dev2[1])

['Exfumadora', 'desde', 'hace', '9', 'años', '.']
['Exfumadora', 'desde', 'hace', '9', 'años', '.']


##### **Test set**

In [ ]:
# TEST SET

# *** without considering clinical cases independently ***

# sentences
getter_test = sentence_test(df_data_test)
sentences_bert_test = [[s[0] for s in sentence] for sentence in getter_test.sentences]
print(sentences_bert_test[0])

# start char position
start_bert_test = [[s[2] for s in sentence] for sentence in getter_test.sentences]
print(start_bert_test[0])

# ----------------

# *** By clinical case ***
getter_by_cc_test = df_data_test.groupby("File_Index").apply(sentence_test)

sentences_bert_test_by_cc = []
start_bert_test_by_cc = []

for getter_i in getter_by_cc_test: # iterating over all the files
  # sentences
  sent_by_cc = [[s[0] for s in sentence] for sentence in getter_i.sentences]
  sentences_bert_test_by_cc.append(sent_by_cc)
  # start char position
  start_by_cc = [[s[2] for s in sentence] for sentence in getter_i.sentences]
  start_bert_test_by_cc.append(start_by_cc)

print()
print(sentences_bert_test_by_cc[0][0])
print(start_bert_test_by_cc[0][0])

['Paciente', 'mujer', ',', '75', 'años', 'consulta', 'el', '4-6-2003', ',', 'refiriendo', 'como', 'antecedentes', 'personales', ':', 'Alergia', 'a', 'salicilatos', '.']
[0, 9, 14, 16, 19, 24, 33, 36, 44, 46, 57, 62, 75, 85, 87, 95, 97, 108]

['Paciente', 'mujer', ',', '75', 'años', 'consulta', 'el', '4-6-2003', ',', 'refiriendo', 'como', 'antecedentes', 'personales', ':', 'Alergia', 'a', 'salicilatos', '.']
[0, 9, 14, 16, 19, 24, 33, 36, 44, 46, 57, 62, 75, 85, 87, 95, 97, 108]


In [ ]:
print(sentences_bert_test_by_cc[0][1])
print(sentences_bert_test[1])

['A', 'los', '59', 'años', 'fué', 'diagnosticada', 'de', 'fiebre', 'de', 'probable', 'etiología', 'específica', ',', 'tratada', 'con', 'tuberculostáticos', ',', 'según', 'pauta', 'habitual', '.']
['A', 'los', '59', 'años', 'fué', 'diagnosticada', 'de', 'fiebre', 'de', 'probable', 'etiología', 'específica', ',', 'tratada', 'con', 'tuberculostáticos', ',', 'según', 'pauta', 'habitual', '.']


#### **Bert tokenizer**

The following command constructs a **BERT tokenizer**. The Bert implementation comes with a pretrained tokenizer and a definsed vocabulary. We load the one related to the smallest pre-trained model bert-base-cased. We use the cased variate since it is well suited for NER.

NOTE: we are not converting all to lower case

In [ ]:
 # The Bert implementation comes with a pretrained tokenizer and a definied vocabulary
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False) # beto
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
#tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, start_offset):
    tokenized_sentence = []
    labels = []
    start_pos = []

    for word, label, start in zip(sentence, text_labels, start_offset): 
      # start offset contains the position of the first character on every word

        # Tokenize the word and count # of subwords the word is broken into
        if str(word) == 'nan':
          tokenized_word = 'UNK'
          n_subwords = 1

        else:
          tokenized_word = tokenizer.tokenize(word)
          n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

        # Add the same start character position
        start_pos.extend([start] * n_subwords)

    return tokenized_sentence, labels, start_pos

# this code breaks up each word into several tokens for every sentence and preserves
# the tag ("O", "B-MOR", "I-MOR", "E-MOR", "S-MOR", "V-MOR") while dong it.
# This way, the sentences become longer (are formed by a higher number of tokens than before)
# this is done to find words that could be present in different forms while having the same meaning

In [ ]:
def tokenize_test(sentence, start_offset):
    tokenized_sentence = []
    start_pos = []

    for word, start in zip(sentence, start_offset):
        # Tokenize the word and count # of subwords the word is broken into
        if str(word) == 'nan':
          tokenized_word = 'UNK'
          n_subwords = 1
        else:
          tokenized_word = tokenizer.tokenize(word)
          n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same start character position
        start_pos.extend([start] * n_subwords)

    return tokenized_sentence, start_pos

In [ ]:
sentences = " ".join([" ".join([str(s) for s in sent]) for sent in sentences_bert_test_by_cc[0]])

In [ ]:
sentences

'Paciente mujer , 75 años consulta el 4-6-2003 , refiriendo como antecedentes personales : Alergia a salicilatos . A los 59 años fué diagnosticada de fiebre de probable etiología específica , tratada con tuberculostáticos , según pauta habitual . En 1.998 , es diagnosticada de fiebre mediterránea familiar después de exclusión de otros procesos . Osteoporosis . Está en tratamiento con Colchicina ( 0,5 mg / día),Aines ( 50 mg / día ) , deflazacor ( 6 mg / día ) y Omeprazol . \n Historia actual : La paciente tuvo una caída casual en octubre de 2.002 sobre flanco izquierdo presentando 10 días después un episodio de hematuria monosintomática . En mayo del 2.003 , tuvo fiebre y dolor lumbar izquierdo ingresando , en un centro hospitalario , donde se realiza un TAC abdominal que se informa como hipernefroma izquierdo con riñón derecho normal . Se le trata con quinolonas . No se realiza cultivo . En el momento de la consulta se encuentra asintomática , sin fiebre , ni dolor lumbar . \n Explora

##### **Tokenize Train and Development sets**

In [ ]:
print("Number of words in the first sentence: %d" %len(sentences_bert_train[0]))

# 1. Tokenize each sentence and keep text and labels
tokenized_texts_and_labels_train = [tokenize_and_preserve_labels(sent, labs,start) for sent, labs, start 
                                    in zip(sentences_bert_train, labels_bert_train, start_bert_train)]
print("Number of words in the first sentence after tokenizing: %d" %len(tokenized_texts_and_labels_train[0][0]))
print(tokenized_texts_and_labels_train[0][0]) # first sentence; text part
print(tokenized_texts_and_labels_train[0][1]) # first sentence; label part (Contains both the sentences and the labels (tags))
print(tokenized_texts_and_labels_train[0][2]) # first sentence; start position part (Contains both the sentences and the labels (tags))


# 2. Split each tokenized sentence into text and labels
tokenized_texts_train = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_train]
labels_train = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_train]
start_pos_train = [token_label_pair[2] for token_label_pair in tokenized_texts_and_labels_train]

print()
print(tokenized_texts_train[0])


# 3. Cut and pad the token and label sequences to the desired length. (max_len: max sequence length = 75 tokens)
input_ids_train = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_train],
                          maxlen=max_len, dtype="long", value=0.0,
                          truncating="post", padding="post")

tags_train = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_train],
                     maxlen=max_len, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

# tokenizer.convert_tokens_to_ids(txt) is a built-in function that converts a token string 
# (or a sequence of tokens) into a single integer id (or a sequence of ids),
# using the pre-defined vocabulary in tokenizer.
print("\nID representation of the first sentence: \n %s" %input_ids_train[0])


# 4. Creation of attention masks to IGNORE certain words (padded elements)
# If the ID of a word is 0, that word is "PAD", a mask must be created to IGNORE such word (to ignore padded elements)
attention_masks_train = [[float(i != 0.0) for i in ii] for ii in input_ids_train] # it's an array of arrays
print("\nAttention mask of the first sentence: \n %s" %attention_masks_train[0])


# 5. Split train dataset into train and validation
train_inputs, valid_inputs, train_tags, valid_tags = train_test_split(input_ids_train, tags_train, random_state=0, test_size=0.2)
train_masks, valid_masks, _, _ = train_test_split(attention_masks_train, input_ids_train, random_state=0, test_size=0.2)
# _: used to not store tags of masked words

Number of words in the first sentence: 24
Number of words in the first sentence after tokenizing: 43
['AN', '##AM', '##NE', '##SI', '##S', 'Mujer', 'de', '67', 'años', 'con', 'ante', '##cedent', '##es', 'personales', 'de', 'hip', '##oti', '##roi', '##dis', '##mo', 'en', 'tratamiento', 'con', 'le', '##vot', '##iro', '##xin', '##a', 'y', 'fu', '##mad', '##ora', 'activa', 'de', '12', '.', '5', 'pa', '##quet', '##es', '/', 'año', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[0, 0, 0, 0, 0, 10, 16, 19, 22, 27, 31, 31, 31, 44, 55, 58, 58, 58, 58, 58, 73, 76, 88, 92, 92, 92, 92, 92, 105, 107, 107, 107, 116, 123, 126, 126, 126, 131, 131, 131, 139, 140, 143]

['AN', '##AM', '##NE', '##SI', '##S', 'Mujer', 'de', '67', 'años', 'con', 'ante', '##cedent', '##es', 'personales', 'de', 'hip', '##oti', '##roi', '##dis', '##mo', '

In [ ]:
print(sentences_bert_train[0])

['ANAMNESIS', '\n', 'Mujer', 'de', '67', 'años', 'con', 'antecedentes', 'personales', 'de', 'hipotiroidismo', 'en', 'tratamiento', 'con', 'levotiroxina', 'y', 'fumadora', 'activa', 'de', '12.5', 'paquetes', '/', 'año', '.']


['ANAMNESIS', '\n', 'Mujer', 'de', '67', 'años', 'con', 'antecedentes', 'personales', 'de', 'hipotiroidismo', 'en', 'tratamiento', 'con', 'levotiroxina', 'y', 'fumadora', 'activa', 'de', '12.5', 'paquetes', '/', 'año', '.']


['AN', '##AM', '##NE', '##SI', '##S', 'Mujer', 'de', '67', 'años', 'con', 'ante', '##cedent', '##es', 'personales', 'de', 'hip', '##oti', '##roi', '##dis', '##mo', 'en', 'tratamiento', 'con', 'le', '##vot', '##iro', '##xin', '##a', 'y', 'fu', '##mad', '##ora', 'activa', 'de', '12', '.', '5', 'pa', '##quet', '##es', '/', 'año', '.']

 [ 50972  36535  93280  44802  10731  95391  10104  12316  11278  10173
  15865 104101  10171  90770  10104  25377  23841  56859  17442  11033
  10110  56708  10173  10141  63129  14213  76750  10113    193  11005
  42998  14945  50796  10104  10186    119    126  10931  27579  10171
    120  11734    119      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


**Tokenize development set**

In [ ]:
# *** without considering clinical cases independently ***

print("Number of words in the first sentence: %d" %len(sentences_bert_dev[0]))

# 1. Tokenize each sentence and keep text and labels
tokenized_texts_and_labels_dev = [tokenize_and_preserve_labels(sent, labs,start) for sent, labs,start 
                                  in zip(sentences_bert_dev, labels_bert_dev,start_bert_dev)]
print("Number of words in the first sentence after tokenizing: %d" %len(tokenized_texts_and_labels_dev[0][0]))
print(tokenized_texts_and_labels_dev[0][0]) # first sentence; text part
print(tokenized_texts_and_labels_dev[0][1]) # first sentence; label part (Contains both the sentences and the labels (tags))
print(tokenized_texts_and_labels_dev[0][2]) # first sentence; start char position part (Contains both the sentences and the labels (tags))


# 2. Split each tokenized sentence into text and labels
tokenized_texts_dev = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_dev]
labels_dev = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_dev]
start_pos_dev = [token_label_pair[2] for token_label_pair in tokenized_texts_and_labels_dev]

print()
print(tokenized_texts_dev[0])


# 3. Cut and pad the token and label sequences to the desired length. (max_len: max sequence length = 75 tokens)
input_ids_dev = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_dev],
                          maxlen=max_len, dtype="long", value=0.0,
                          truncating="post", padding="post")

tags_dev = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_dev],
                     maxlen=max_len, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")
print("\nID representation of the first sentence: \n %s" %input_ids_dev[0])

start_dev = pad_sequences(start_pos_dev,
                     maxlen=max_len, value=-1, padding="post", truncating="post")
print("\nStart char positions the first sentence: \n %s" %start_dev[0])


# 4. Creation of attention masks to IGNORE certain words (padded elements)
attention_masks_dev = [[float(i != 0.0) for i in ii] for ii in input_ids_dev] # it's an array of arrays
print("\nAttention mask of the first sentence: \n %s" %attention_masks_dev[0])


# 5. Split train dataset into train and validation
dev_inputs = input_ids_dev
dev_tags =  tags_dev
dev_masks = attention_masks_dev
# _: used to not store tags of masked words

Number of words in the first sentence: 43
Number of words in the first sentence after tokenizing: 70
['Ana', '##mne', '##sis', 'Var', '##ón', 'de', '74', 'años', ',', 'ex', '##fum', '##ador', 'desde', 'hace', '15', 'años', ',', 'con', 'único', 'ante', '##cedent', '##e', 'de', 'hip', '##erten', '##sión', ',', 'dis', '##lip', '##emia', 'y', 'ap', '##endi', '##ce', '##cto', '##mi', '##zado', ';', 'se', 'diagnostic', '##a', 'en', 'marzo', 'de', '2013', 'de', 'car', '##cino', '##ma', 'de', 'células', 'trans', '##icion', '##ales', 'de', 'vej', '##iga', 'E', '-', 'IV', '(', 'pu', '##lm', '##onare', '##s', 'y', 'ós', '##eas', ')', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MOR', 'B-MOR', 'B-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'I-MOR', 'E-MOR'

In [ ]:
num_files_dev = len(sentences_bert_dev_by_cc) 
print("Number of files in the development set: %d" %num_files_dev)
len_sent_by_cc = [len(cc) for cc in sentences_bert_dev_by_cc] # number of sentences in each file
sentences_bert_dev_by_cc2 = []
offset = 0

for length_file in len_sent_by_cc:
  sent_cc = sentences_bert_dev[offset: offset + length_file]
  sentences_bert_dev_by_cc2.append(sent_cc)
  offset = offset + length_file

Number of files in the development set: 250


In [ ]:
# *** By clinical case ***

len_sent_by_cc = [len(cc) for cc in sentences_bert_dev_by_cc] # number of sentences in each file

dev_inputs_by_cc = []
dev_tags_by_cc = []
dev_masks_by_cc = []
start_dev_by_cc = []
offset = 0

for length_file in len_sent_by_cc:
  dev_inputs_by_cc.append(dev_inputs[offset: offset + length_file])
  dev_tags_by_cc.append(dev_tags[offset: offset + length_file])
  dev_masks_by_cc.append(dev_masks[offset: offset + length_file])
  start_dev_by_cc.append(start_dev[offset: offset + length_file])

  offset = offset + length_file

In [ ]:
print(tag2idx)

{'E-MOR': 0, 'O': 1, 'I-MOR': 2, 'B-MOR': 3, 'S-MOR': 4, 'V-MOR': 5, 'PAD': 6}


In [ ]:
print(dev_inputs[26])
print()
print(dev_inputs_by_cc[1][1])
print("\n-----------------------------------------------------------------------\n")
print(dev_tags[26])
print()
print(dev_tags_by_cc[1][1])
print("\n-----------------------------------------------------------------------\n")
print(dev_masks[26])
print()
print(dev_masks_by_cc[1][1])

[ 34387  11488  10567  10104  10671 100076  10107  48602  11272  10110
  16814  10104  10185  10110  10109  86153  10104 103174  14865  10173
  12596  76456  10165  81270  11389    119      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0]

[ 34387  11488  10567  10104  10671 100076  10107  48602  11272  10110
  16814  10104  10185  10110  10109  86153  10104 103174  14865  10173
  12596  76456  10165  81270  11389    119      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0    

**Tokenize development set 2**

In [ ]:
# *** without considering clinical cases independently ***

print("Number of words in the first sentence: %d" %len(sentences_bert_dev2[0]))

# 1. Tokenize each sentence and keep text and labels
tokenized_texts_and_labels_dev2 = [tokenize_and_preserve_labels(sent, labs, start) for sent, labs,start 
                                   in zip(sentences_bert_dev2, labels_bert_dev2,start_bert_dev2)]
print("Number of words in the first sentence after tokenizing: %d" %len(tokenized_texts_and_labels_dev2[0][0]))
print(tokenized_texts_and_labels_dev2[0][0]) # first sentence; text part
print(tokenized_texts_and_labels_dev2[0][1]) # first sentence; label part (Contains both the sentences and the labels (tags))
print(tokenized_texts_and_labels_dev2[0][2]) # first sentence; start char position part (Contains both the sentences and the labels (tags))


# 2. Split each tokenized sentence into text and labels
tokenized_texts_dev2 = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_dev2]
labels_dev2 = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_dev2]
start_pos_dev2 = [token_label_pair[2] for token_label_pair in tokenized_texts_and_labels_dev2]

print()
print(tokenized_texts_dev2[0])


# 3. Cut and pad the token and label sequences to the desired length. (max_len: max sequence length = 75 tokens)
input_ids_dev2 = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_dev2],
                          maxlen=max_len, dtype="long", value=0.0,
                          truncating="post", padding="post")

tags_dev2 = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_dev2],
                     maxlen=max_len, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")
print("\nID representation of the first sentence: \n %s" %input_ids_dev2[0])

start_dev2 = pad_sequences(start_pos_dev2,
                     maxlen=max_len, value=-1, padding="post", truncating="post")
print("\nStart char positions the first sentence: \n %s" %start_dev2[0])

# 4. Creation of attention masks to IGNORE certain words (padded elements)
attention_masks_dev2 = [[float(i != 0.0) for i in ii] for ii in input_ids_dev2] # it's an array of arrays
print("\nAttention mask of the first sentence: \n %s" %attention_masks_dev2[0])


# 5. Split train dataset into train and validation
dev_inputs2 = input_ids_dev2
dev_tags2 =  tags_dev2
dev_masks2 = attention_masks_dev2
# _: used to not store tags of masked words

Number of words in the first sentence: 35
Number of words in the first sentence after tokenizing: 51
['Ana', '##mne', '##sis', 'Mujer', 'de', '60', 'años', 'diagnostic', '##ada', 'de', 'enfermedad', 'pu', '##lm', '##onar', 'ob', '##stru', '##ctiva', 'c', '##rónica', 'y', 'tras', '##torno', 'bi', '##pola', '##r', 'de', 'años', 'de', 'evolución', ',', 'en', 'tratamiento', 'con', 'car', '##ba', '##maz', '##ep', '##ina', 'hasta', 'la', 'fecha', ',', 'sin', 'otros', 'ante', '##cedent', '##es', 'personales', 'de', 'interés', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[0, 0, 0, 10, 16, 19, 22, 27, 27, 41, 44, 55, 55, 55, 64, 64, 64, 76, 76, 84, 86, 86, 96, 96, 96, 104, 107, 112, 115, 124, 126, 129, 141, 145, 145, 145, 145, 145, 159, 165, 168, 173, 175, 179, 185, 185, 185, 198, 2

In [ ]:
# *** By clinical case ***

len_sent_by_cc2 = [len(cc) for cc in sentences_bert_dev_by_cc2] # number of sentences in each file

dev_inputs_by_cc2 = []
dev_tags_by_cc2 = []
dev_masks_by_cc2 = []
start_dev_by_cc2 = []
offset = 0

for length_file in len_sent_by_cc2:
  dev_inputs_by_cc2.append(dev_inputs2[offset: offset + length_file])
  dev_tags_by_cc2.append(dev_tags2[offset: offset + length_file])
  dev_masks_by_cc2.append(dev_masks2[offset: offset + length_file])
  start_dev_by_cc2.append(start_dev2[offset: offset + length_file])


  offset = offset + length_file

##### **Tokenize test set**

In [ ]:
# *** without considering clinical cases independently ***

print("Number of words in the first sentence: %d" %len(sentences_bert_test[0]))

# 1. Tokenize each sentence 
tokenized_texts_and_labels_test = [tokenize_test(sent,start) for sent, start in zip(sentences_bert_test,start_bert_test)]
print("Number of words in the first sentence after tokenizing: %d" %len(tokenized_texts_and_labels_test[0][0]))
print(tokenized_texts_and_labels_test[0][0]) # first sentence; text part
print(tokenized_texts_and_labels_test[0][1]) # first sentence; start char position part (Contains both the sentences and the labels (tags))


# 2. Split each tokenized sentence into text and start char position
tokenized_texts_test = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_test]
start_pos_test = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_test]
print()
print(tokenized_texts_test[0])

# 3. Cut and pad the token and label sequences to the desired length. (max_len: max sequence length = 75 tokens)
input_ids_test= pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=max_len, dtype="long", value=0.0,
                          truncating="post", padding="post")
print("\nID representation of the first sentence: \n %s" %input_ids_test[0])

start_test = pad_sequences(start_pos_test,
                     maxlen=max_len, value=-1, padding="post", truncating="post")
print("\nStart char positions the first sentence: \n %s" %start_test[0])

# 4. Creation of attention masks to IGNORE certain words (padded elements)
attention_masks_test = [[float(i != 0.0) for i in ii] for ii in input_ids_test] # it's an array of arrays
print("\nAttention mask of the first sentence: \n %s" %attention_masks_test[0])


# 5. Split train dataset into train and validation
test_inputs = input_ids_test
test_masks = attention_masks_test
# _: used to not store tags of masked words

Number of words in the first sentence: 18
Number of words in the first sentence after tokenizing: 31
['Pac', '##iente', 'mujer', ',', '75', 'años', 'consulta', 'el', '4', '-', '6', '-', '2003', ',', 'ref', '##iri', '##endo', 'como', 'ante', '##cedent', '##es', 'personales', ':', 'Ale', '##rgia', 'a', 'sal', '##ici', '##lato', '##s', '.']
[0, 0, 9, 14, 16, 19, 24, 33, 36, 36, 36, 36, 36, 44, 46, 46, 46, 57, 62, 62, 62, 75, 85, 87, 87, 95, 97, 97, 97, 97, 108]

['Pac', '##iente', 'mujer', ',', '75', 'años', 'consulta', 'el', '4', '-', '6', '-', '2003', ',', 'ref', '##iri', '##endo', 'como', 'ante', '##cedent', '##es', 'personales', ':', 'Ale', '##rgia', 'a', 'sal', '##ici', '##lato', '##s', '.']

ID representation of the first sentence: 
 [ 82376  22847  24033    117  11417  11278  86153  10125    125    118
    127    118  10295    117  48056  19334  17560  10225  15865 104101
  10171  90770    131  59340  85368    169  31119  13439  35304  10107
    119      0      0      0      0     

In [ ]:
num_files_test = len(sentences_bert_test_by_cc) 
print("Number of files in the test set: %d" %num_files_test)
len_sent_by_cc = [len(cc) for cc in sentences_bert_test_by_cc] # number of sentences in each file
sentences_bert_test_by_cc2 = []
offset = 0

for length_file in len_sent_by_cc:
  sent_cc = sentences_bert_test[offset: offset + length_file]
  sentences_bert_test_by_cc2.append(sent_cc)
  offset = offset + length_file


# *** By clinical case ***

len_sent_by_cc = [len(cc) for cc in sentences_bert_test_by_cc] # number of sentences in each file

test_inputs_by_cc = []
test_masks_by_cc = []
start_test_by_cc = []
offset = 0

for length_file in len_sent_by_cc:
  test_inputs_by_cc.append(test_inputs[offset: offset + length_file])
  test_masks_by_cc.append(test_masks[offset: offset + length_file])
  start_test_by_cc.append(start_test[offset: offset + length_file])

  offset = offset + length_file

Number of files in the test set: 5232


In [ ]:
print(test_inputs[48])
print()
print(test_inputs_by_cc[1][1])
print("\n-----------------------------------------------------------------------\n")
print(test_masks[48])
print()
print(test_masks_by_cc[1][1])
print("\n-----------------------------------------------------------------------\n")
print(start_test[48])
print()
print(start_test_by_cc[1][1])

[10243 10182 52909   117 15882 10537 35076 10567 25183 10104 10109 93622
 48832 39467 10280   113 29137   119   126   181 10350 53166   119   120
 10233 10237   119     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]

[10243 10182 52909   117 15882 10537 35076 10567 25183 10104 10109 93622
 48832 39467 10280   113 29137   119   126   181 10350 53166   119   120
 10233 10237   119     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]

-----------------------------------------------------------------------

[1.0, 1.0

#### **Convert data into tensors**

##### **Train and development sets**

**Train set**

In [ ]:
# Convert to TENSORS (working with pytorch)

# text
train_inputs = torch.tensor(train_inputs)
valid_inputs = torch.tensor(valid_inputs)

# tags
train_tags = torch.tensor(train_tags)
valid_tags = torch.tensor(valid_tags)

# masks
train_masks = torch.tensor(train_masks)
valid_masks = torch.tensor(valid_masks)


# Define DATALOADERS
# shuffle the data of train set with RandomSampler 
train_data = TensorDataset(train_inputs, train_masks, train_tags)
train_sampler = RandomSampler(train_data, ) # it just shuffles wihtout selecting a subset since num_samples = None
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

valid_data = TensorDataset(valid_inputs, valid_masks, valid_tags)
valid_sampler = SequentialSampler(valid_data) # validation data is passed sequentially
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

**Development set**

In [ ]:
# Convert to TENSORS (working with pytorch)

# text
dev_inputs = torch.tensor(dev_inputs)

# tags
dev_tags = torch.tensor(dev_tags)

# masks
dev_masks = torch.tensor(dev_masks)


# Define DATALOADERS

# *** without considering clinical cases individually ***
dev_data = TensorDataset(dev_inputs, dev_masks, dev_tags)
dev_sampler = SequentialSampler(dev_data) # validation data is passed sequentially
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

# *** by clinical case***

**Development set 2**

In [ ]:
# Convert to TENSORS (working with pytorch)

# text
dev_inputs2 = torch.tensor(dev_inputs2)

# tags
dev_tags2 = torch.tensor(dev_tags2)

# masks
dev_masks2 = torch.tensor(dev_masks2)


# Define DATALOADERS

# *** without considering clinical cases individually ***
dev_data2 = TensorDataset(dev_inputs2, dev_masks2, dev_tags2)
dev_sampler2 = SequentialSampler(dev_data2) # validation data is passed sequentially
dev_dataloader2 = DataLoader(dev_data2, sampler=dev_sampler2, batch_size=batch_size)

# *** by clinical case***

##### **Test set**

In [ ]:
# Convert to TENSORS (working with pytorch)

# text
test_inputs = torch.tensor(test_inputs)

# masks
test_masks = torch.tensor(test_masks)


# Define DATALOADERS

# *** without considering clinical cases individually ***
test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data) # validation data is passed sequentially
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


#### **STAGE 1: BERT Classification pre-trained model**

**BertForTokenClassification class** from transformers is used for token-level predictions. BertForTokenClassification is a fine-tuning model that wraps BertModel and adds token-level classifier on top of the BertModel. 

The token-level classifier is a linear layer that takes as input the last hidden state of the sequence. We load the pre-trained bert-base-cased model and provide the number of possible labels.

In [ ]:
# loading the pretrained model of BERT base 

model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

# pass the model parameters to the GPU
model.cuda();

#### **STAGE 2: Fine-tuning**

Prior to the fine-tuning process, the optimizer (**AdamW**, Implements Adam algorithm with weight decay fix) and parameters to update must be setup. In order to avoid overfitting, some regularization will be applied by including weight_decay to the main weight matrices. 

If you have limited resources, you can also try to just train the linear classifier on top of BERT and keep all other weights fixed. This will still give you a good performance.

The **AdamW** optimizer has parameters:
- lr (float) – learning rate. Default 1e-3.
- betas (tuple of 2 floats) – Adams beta parameters (b1, b2). Default: (0.9, 0.999)
- eps (float) – Adams epsilon. Default: 1e-6
- weight_decay (float) – Weight decay. Default: 0.0
- correct_bias (bool) – can be set to False to avoid correcting bias in Adam (e.g. like in Bert TF repository). Default True.

In [ ]:
FULL_FINETUNING = True

if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5, # learning rate
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler. Used to linearly reduce the learning rate throughout the epochs.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

#### **STAGE 3: Fit BERT for named entity recognition**

f1_score from the seqeval, accuracy is measured on token level.

In [ ]:
!pip install seqeval
import seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=10ff880c22ad9bbd48d14f88ed3745f1038cc710bdfc5319ae5071ae4ef1c817
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


#### **Train and evaluate the BERT model**

In [ ]:
from seqeval.metrics import classification_report

In [ ]:
from seqeval.metrics import f1_score, accuracy_score
from tqdm import tnrange as trange

## Store the average loss after each epoch to plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        model.zero_grad() # reset gradients before backward pass

        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()

        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()

    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []

    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
            
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Compute the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print()


In [ ]:
f1_score(pred_tags, valid_tags)

#### **Function definitions**

In [ ]:
def confusion_matrix(actual, predicted):
    #classes       = np.unique(np.concatenate((actual,predicted)))
    classes = ['B-MOR', 'I-MOR', 'E-MOR', 'S-MOR', 'V-MOR', 'O']
    confusion_mtx = np.empty((len(classes),len(classes)),dtype=np.int)
    for i,a in enumerate(classes):
        for j,p in enumerate(classes):
            value = sum([sum([np.where((actual[sent][word]==a)*(predicted[sent][word]==p))[0].shape[0] 
                              for word in range(len(actual[sent]))]) for sent in range(len(actual))])
            #confusion_mtx[i,j] = sum([np.where((actual[sent]==a)*(predicted[sent]==p))[0].shape[0] for sent in range(len(actual))])
            confusion_mtx[i,j] = value
    return confusion_mtx

In [ ]:
# code to extract METRICS FOR ENTITY AND NON-ENTITY from the confusion matrix

def metrics_from_cm(cm,labels):
  TP = [[v for v in value[0:len(labels)]] for value in cm[0:len(labels)]]
  print(TP)
  TP = np.array(TP)
  TP = sum(sum(TP))

  FN = [value[-1] for value in cm[0:len(labels)]] # last column is O
  print(FN)
  FN = np.array(FN) 
  FN = sum(FN)

  FP = cm[len(labels)][0:len(labels)] # last column is O
  print(FP)
  FP = sum(FP)

  TN = cm[len(labels)][len(labels)] # last column is O
  print(TN)

  return TP, FN, FP, TN

#### **STAGE 4: Apply the fine-tuned model over the validation set**

In [ ]:
labels = ['B-MOR', 'I-MOR', 'E-MOR', 'S-MOR', 'V-MOR']

In [ ]:
report_valid = flat_classification_report(y_true = [valid_tags], y_pred = [pred_tags], labels = labels)
print(report_valid)

In [ ]:
def confusion_matrix_no_sent(actual, predicted):
    #classes       = np.unique(np.concatenate((actual,predicted)))
    classes = ['B-MOR', 'I-MOR', 'E-MOR', 'S-MOR', 'V-MOR', 'O']
    confusion_mtx = np.empty((len(classes),len(classes)),dtype=np.int)
    for i,a in enumerate(classes):
        for j,p in enumerate(classes):
            value = sum([np.where((actual[word]==a)*(predicted[word]==p))[0].shape[0] 
                              for word in range(len(actual))]) 
            #confusion_mtx[i,j] = sum([np.where((actual[sent]==a)*(predicted[sent]==p))[0].shape[0] for sent in range(len(actual))])
            confusion_mtx[i,j] = value
    return confusion_mtx

In [ ]:
# not done by clinical case
actual    = np.array(valid_tags)
predicted = np.array(pred_tags)
confusion_matrix_valid = confusion_matrix_no_sent(actual,predicted)

df = pd.DataFrame(confusion_matrix_valid, columns = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'], index = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'])
df

In [ ]:
TP, FN, FP, TN = metrics_from_cm(confusion_matrix_valid,labels)
print(TP)
print(FN)
print(FP)
print(TN)

#### **STAGE 5: Apply the fine-tuned model over the development set**

In [ ]:
def tokens_to_words(tokens_sent,label_indx_sent, true_label_indx_sent, start_char_pos, tag_values):
  new_tok, new_lab, true_lab, new_start_pos = [], [], [], []

  for tokens, label_indices, true_label_indices, start_chars in zip(tokens_sent, label_indx_sent, true_label_indx_sent, start_char_pos):
    new_tok_aux, new_lab_aux, true_lab_aux, new_start_pos_aux = [], [], [], []
    for token, label_idx, true_label_idx, start_char_i in zip(tokens, label_indices,true_label_indices,start_chars):
      if tag_values[true_label_idx] != "PAD":
        if token.startswith("##"):
            new_tok_aux[-1] = new_tok_aux[-1] + token[2:]
        else:
            new_lab_aux.append(tag_values[label_idx])
            true_lab_aux.append(tag_values[true_label_idx])
            new_tok_aux.append(token)
            new_start_pos_aux.append(start_char_i)

    new_lab.append(new_lab_aux)
    true_lab.append(true_lab_aux)
    new_tok.append(new_tok_aux)
    new_start_pos.append(new_start_pos_aux)

  return new_tok, new_lab, true_lab, new_start_pos

In [ ]:
def tokens_to_words_test(tokens_sent,label_indx_sent, start_char_pos, tag_values):
  new_tok, new_lab, new_start_pos = [], [], []

  for tokens, label_indices, start_chars in zip(tokens_sent, label_indx_sent, start_char_pos):
    new_tok_aux, new_lab_aux, new_start_pos_aux = [], [], []
    for token, label_idx, start_char_i in zip(tokens, label_indices, start_chars):
      if token != '[PAD]':
        if token.startswith("##"):
            new_tok_aux[-1] = new_tok_aux[-1] + token[2:]
        else:
            new_lab_aux.append(tag_values[label_idx])
            new_tok_aux.append(token)
            new_start_pos_aux.append(start_char_i)

    new_lab.append(new_lab_aux)
    new_tok.append(new_tok_aux)
    new_start_pos.append(new_start_pos_aux)

  return new_tok, new_lab, new_start_pos

In [ ]:
# labels used in the classification report
labels = ['B-MOR', 'I-MOR', 'E-MOR', 'S-MOR', 'V-MOR']

In [ ]:
#tokenized_sentence_dev = tokenizer.encode(sentences_bert_dev[0])
#input_ids_dev2 = torch.tensor([tokenized_sentence_dev]).cuda()

In [ ]:
#employ: (already defined)

# text: dev_inputs = torch.tensor(dev_inputs)

# tags: dev_tags = torch.tensor(dev_tags)

# masks: dev_masks = torch.tensor(dev_masks)

**Predictions over development set**

The predictions over the development set are obtained by clinical case. Therefore, loop over all clinical cases.

In [ ]:
# loop over all the clinical cases because all do not fit in memory

#input_ids_dev2 = torch.tensor(input_ids_dev[0:10]).cuda()
label_indices_sent_cc = []
tokens_sent_cc = []
for cc in range(len(dev_inputs_by_cc)):
  dev_in = torch.tensor(dev_inputs_by_cc[cc]).cuda()
  dev_m = torch.tensor(dev_masks_by_cc[cc]).cuda()
  with torch.no_grad(): # we don't want to optimize
    output = model(dev_in, token_type_ids = None, attention_mask = dev_m)
    #logits = output[1].detach().cpu().numpy()
  label_indices_sent_cc.append(np.argmax(output[0].to('cpu').numpy(), axis=2))
  #label_indices_sent_cc.append([list(p) for p in np.argmax(logits, axis=2)])

# convert input sequence into tokens
  tokens_sent_cc.append([tokenizer.convert_ids_to_tokens(ids.to('cpu').numpy()) for ids in dev_in])


In [ ]:
new_tokens_cc, new_labels_cc, true_labels_cc, new_start_pos_cc = [], [], [], []
new_tokens_all, new_labels_all, true_labels_all, new_start_pos_all = [], [], [], []

#dev_tags_by_cc contains the true labels of each token

for cc in range(len(dev_inputs_by_cc)):
  new_tokens, new_labels, true_labels, new_start_pos = [], [], [], []
  new_tokens, new_labels, true_labels, new_start_pos = tokens_to_words(tokens_sent_cc[cc], 
                                        label_indices_sent_cc[cc], dev_tags_by_cc[cc], 
                                        start_dev_by_cc[cc],tag_values)
  new_tokens_cc.append(new_tokens)
  new_labels_cc.append(new_labels)
  true_labels_cc.append(true_labels)
  new_start_pos_cc.append(new_start_pos)

  new_tokens_all.extend(new_tokens)
  new_labels_all.extend(new_labels)
  true_labels_all.extend(true_labels)
  new_start_pos_all.extend(new_start_pos)

In [ ]:
!ls 'drive/My Drive/Ejemplos NER - TFM/results/dev/'

cc_onco100.ann	cc_onco106.ann	new_labels_cc	  new_tokens_cc
cc_onco101.ann	ex2.ann		new_start_pos_cc  true_labels_cc


In [ ]:
import pickle as pkl

with open(path+'results/dev/new_tokens_cc', 'wb') as file: 
  pkl.dump(new_tokens_cc, file)
file.close()

with open(path+'results/dev/new_labels_cc', 'wb') as file: 
  pkl.dump(new_labels_cc, file)
file.close()

with open(path+'results/dev/true_labels_cc', 'wb') as file: 
  pkl.dump(true_labels_cc, file)
file.close()

with open(path+'results/dev/new_start_pos_cc', 'wb') as file: 
  pkl.dump(new_start_pos_cc, file)
file.close()

In [ ]:
len(new_start_pos_cc)

250

In [ ]:
# Example:
for token, label, true_label, new_start in zip(new_tokens_cc[0][0], new_labels_cc[0][0], 
                                               true_labels_cc[0][0], new_start_pos_cc[0][0]):
    print("{}\t{}\t{}\t{}".format(true_label, label, token,new_start))

O	O	Anamnesis	0
O	O	Varón	10
O	O	de	16
O	O	74	19
O	O	años	22
O	O	,	26
O	O	exfumador	28
O	O	desde	38
O	O	hace	44
O	O	15	49
O	O	años	52
O	O	,	56
O	O	con	58
O	O	único	62
O	O	antecedente	68
O	O	de	80
O	O	hipertensión	83
O	O	,	95
O	O	dislipemia	97
O	O	y	108
O	O	apendicectomizado	110
O	O	;	127
O	O	se	129
O	O	diagnostica	132
O	O	en	144
O	O	marzo	147
O	O	de	153
O	O	2013	156
O	O	de	161
B-MOR	B-MOR	carcinoma	164
I-MOR	I-MOR	de	174
I-MOR	I-MOR	células	177
I-MOR	I-MOR	transicionales	185
I-MOR	I-MOR	de	200
I-MOR	I-MOR	vejiga	203
I-MOR	I-MOR	E	210
I-MOR	I-MOR	-	210
I-MOR	E-MOR	IV	210
E-MOR	O	(	215
O	O	pulmonares	216
O	O	y	227
O	O	óseas	229
O	O	)	234
O	O	.	235


In [ ]:
report_dev = flat_classification_report(y_true = true_labels_all, y_pred = new_labels_all, labels = labels)
print(report_dev)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-MOR       0.82      0.79      0.81      1525
       I-MOR       0.78      0.68      0.73      2420
       E-MOR       0.73      0.71      0.72      1550
       S-MOR       0.90      0.90      0.90      1613
       V-MOR       0.00      0.00      0.00        18

   micro avg       0.81      0.76      0.78      7126
   macro avg       0.65      0.62      0.63      7126
weighted avg       0.80      0.76      0.78      7126



In [ ]:
# not done by clinical case
actual    = np.array(true_labels_all)
predicted = np.array(new_labels_all)
confusion_matrix_dev = confusion_matrix(actual,predicted)

df = pd.DataFrame(confusion_matrix_dev, columns = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'], index = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'])
df

,B-MOR,I-MOR,E-MOR,S-MOR,V-MOR,O
B-MOR,1209,11,1,63,0,241
I-MOR,26,1652,125,9,0,608
E-MOR,0,67,1097,30,0,356
S-MOR,46,0,25,1449,0,93
V-MOR,3,7,3,4,0,1
O,193,383,246,60,0,208101


In [ ]:
TP, FN, FP, TN = metrics_from_cm(confusion_matrix_dev,labels)
print(TP)
print(FN)
print(FP)
print(TN)

[[1209, 11, 1, 63, 0], [26, 1652, 125, 9, 0], [0, 67, 1097, 30, 0], [46, 0, 25, 1449, 0], [3, 7, 3, 4, 0]]
[241, 608, 356, 93, 1]
[193 383 246  60   0]
208101
5827
1299
882
208101


**Predictions over development set 2**

In [ ]:
label_indices_sent_cc2 = []
tokens_sent_cc2 = []
for cc in range(len(dev_inputs_by_cc2)):
  dev_in = torch.tensor(dev_inputs_by_cc2[cc]).cuda()
  dev_m = torch.tensor(dev_masks_by_cc2[cc]).cuda()
  with torch.no_grad(): # we don't want to optimize
    output = model(dev_in, token_type_ids = None, attention_mask = dev_m)
    #logits = output[1].detach().cpu().numpy()
  label_indices_sent_cc2.append(np.argmax(output[0].to('cpu').numpy(), axis=2))
  #label_indices_sent_cc.append([list(p) for p in np.argmax(logits, axis=2)])

# convert input sequence into tokens
  tokens_sent_cc2.append([tokenizer.convert_ids_to_tokens(ids.to('cpu').numpy()) for ids in dev_in])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [ ]:
tag_values

['O', 'V-MOR', 'B-MOR', 'E-MOR', 'I-MOR', 'S-MOR', 'PAD']

In [ ]:
new_tokens_cc, new_labels_cc, true_labels_cc = [], [], []
new_tokens_all, new_labels_all, true_labels_all = [], [], []

#dev_tags_by_cc2 contains the true labels of each token

for cc in range(len(dev_inputs_by_cc2)):
  new_tokens, new_labels, true_labels = [], [], []
  new_tokens, new_labels, true_labels = tokens_to_words(tokens_sent_cc2[cc], 
                                        label_indices_sent_cc2[cc], dev_tags_by_cc2[cc],tag_values)
  new_tokens_cc.append(new_tokens)
  new_labels_cc.append(new_labels)
  true_labels_cc.append(true_labels)

  new_tokens_all.extend(new_tokens)
  new_labels_all.extend(new_labels)
  true_labels_all.extend(true_labels)

In [ ]:
# Example:
for token, label, true_label in zip(new_tokens_cc[1][5], new_labels_cc[1][5], true_labels_cc[1][5]):
    print("{}\t{}\t{}".format(true_label, label, token))

O	O	Diagnóstico
O	O	Por
O	O	tanto
O	O	,
O	O	nos
O	O	encontramos
O	O	ante
O	O	el
O	O	caso
O	O	de
O	O	un
B-MOR	B-MOR	sarcoma
I-MOR	I-MOR	de
E-MOR	E-MOR	Kaposi
O	O	clásico
O	O	del
O	O	adulto
O	O	,
O	O	confirmado
O	O	por
O	O	estudio
O	O	anatomopatológico
O	O	.


In [ ]:
report_dev = flat_classification_report(y_true = true_labels_all, y_pred = new_labels_all, labels = labels)
print(report_dev)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-MOR       0.80      0.86      0.83      1236
       I-MOR       0.77      0.75      0.76      2144
       E-MOR       0.72      0.74      0.73      1264
       S-MOR       0.91      0.91      0.91      1331
       V-MOR       0.00      0.00      0.00         3

   micro avg       0.80      0.80      0.80      5978
   macro avg       0.64      0.65      0.65      5978
weighted avg       0.80      0.80      0.80      5978



In [ ]:
# not done by clinical case
actual    = np.array(true_labels_all)
predicted = np.array(new_labels_all)
confusion_matrix_dev = confusion_matrix(actual,predicted)

df = pd.DataFrame(confusion_matrix_dev, columns = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'], index = ['B-MOR', 'I-MOR','E-MOR','S-MOR','V-MOR','O'])
df

,B-MOR,I-MOR,E-MOR,S-MOR,V-MOR,O
B-MOR,1058,13,1,48,0,116
I-MOR,18,1612,90,3,0,421
E-MOR,1,87,931,22,0,223
S-MOR,48,1,27,1211,0,44
V-MOR,1,0,1,1,0,0
O,199,373,244,47,0,170945


In [ ]:
TP, FN, FP, TN = metrics_from_cm(confusion_matrix_dev,labels)
print(TP)
print(FN)
print(FP)
print(TN)

[[1058, 13, 1, 48, 0], [18, 1612, 90, 3, 0], [1, 87, 931, 22, 0], [48, 1, 27, 1211, 0], [1, 0, 1, 1, 0]]
[116, 421, 223, 44, 0]
[199 373 244  47   0]
170945
5174
804
863
170945


#### **STAGE 6: Final Model**

##### **Tokenize Complete dataset**

In [ ]:
print("Number of words in the first sentence: %d" %len(sentences_bert_complete[0]))

# 1. Tokenize each sentence and keep text and labels
tokenized_texts_and_labels_complete = [tokenize_and_preserve_labels(sent, labs,start) for sent, labs, start 
                                    in zip(sentences_bert_complete, labels_bert_complete, start_bert_complete)]
print("Number of words in the first sentence after tokenizing: %d" %len(tokenized_texts_and_labels_complete[0][0]))
print(tokenized_texts_and_labels_complete[0][0]) # first sentence; text part
print(tokenized_texts_and_labels_complete[0][1]) # first sentence; label part (Contains both the sentences and the labels (tags))
print(tokenized_texts_and_labels_complete[0][2]) # first sentence; start position part (Contains both the sentences and the labels (tags))


# 2. Split each tokenized sentence into text and labels
tokenized_texts_complete = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_complete]
labels_complete = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_complete]
start_pos_complete = [token_label_pair[2] for token_label_pair in tokenized_texts_and_labels_complete]

print()
print(tokenized_texts_complete[0])


# 3. Cut and pad the token and label sequences to the desired length. (max_len: max sequence length = 75 tokens)
input_ids_complete = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_complete],
                          maxlen=max_len, dtype="long", value=0.0,
                          truncating="post", padding="post")

tags_complete = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_complete],
                     maxlen=max_len, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

# tokenizer.convert_tokens_to_ids(txt) is a built-in function that converts a token string 
# (or a sequence of tokens) into a single integer id (or a sequence of ids),
# using the pre-defined vocabulary in tokenizer.
print("\nID representation of the first sentence: \n %s" %input_ids_complete[0])


# 4. Creation of attention masks to IGNORE certain words (padded elements)
# If the ID of a word is 0, that word is "PAD", a mask must be created to IGNORE such word (to ignore padded elements)
attention_masks_complete = [[float(i != 0.0) for i in ii] for ii in input_ids_complete] # it's an array of arrays
print("\nAttention mask of the first sentence: \n %s" %attention_masks_complete[0])


# 5. Split train dataset into train and validation
train_inputs = input_ids_complete
train_tags = tags_complete
train_masks = attention_masks_complete
#train_inputs, valid_inputs, train_tags, valid_tags = train_test_split(input_ids_complete, tags_complete, random_state=0, test_size=0.2)
#train_masks, valid_masks, _, _ = train_test_split(attention_masks_complete, input_ids_complete, random_state=0, test_size=0.2)
# _: used to not store tags of masked words

Number of words in the first sentence: 24
Number of words in the first sentence after tokenizing: 43
['AN', '##AM', '##NE', '##SI', '##S', 'Mujer', 'de', '67', 'años', 'con', 'ante', '##cedent', '##es', 'personales', 'de', 'hip', '##oti', '##roi', '##dis', '##mo', 'en', 'tratamiento', 'con', 'le', '##vot', '##iro', '##xin', '##a', 'y', 'fu', '##mad', '##ora', 'activa', 'de', '12', '.', '5', 'pa', '##quet', '##es', '/', 'año', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[0, 0, 0, 0, 0, 10, 16, 19, 22, 27, 31, 31, 31, 44, 55, 58, 58, 58, 58, 58, 73, 76, 88, 92, 92, 92, 92, 92, 105, 107, 107, 107, 116, 123, 126, 126, 126, 131, 131, 131, 139, 140, 143]

['AN', '##AM', '##NE', '##SI', '##S', 'Mujer', 'de', '67', 'años', 'con', 'ante', '##cedent', '##es', 'personales', 'de', 'hip', '##oti', '##roi', '##dis', '##mo', '

##### **Convert data into tensors**
**Complete dataset**

In [ ]:
# Convert to TENSORS (working with pytorch)

# text
train_inputs = torch.tensor(train_inputs)
#valid_inputs = torch.tensor(valid_inputs)

# tags
train_tags = torch.tensor(train_tags)
#valid_tags = torch.tensor(valid_tags)

# masks
train_masks = torch.tensor(train_masks)
#valid_masks = torch.tensor(valid_masks)


# Define DATALOADERS
# shuffle the data of train set with RandomSampler 
train_data = TensorDataset(train_inputs, train_masks, train_tags)
train_sampler = RandomSampler(train_data, ) # it just shuffles wihtout selecting a subset since num_samples = None
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

#valid_data = TensorDataset(valid_inputs, valid_masks, valid_tags)
#valid_sampler = SequentialSampler(valid_data) # validation data is passed sequentially
#valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

##### **Train final model**

In [ ]:
from seqeval.metrics import f1_score, accuracy_score
from tqdm import tnrange as trange

## Store the average loss after each epoch to plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        model.zero_grad() # reset gradients before backward pass

        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()

        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  import sys


/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Average train loss: 0.0784052338540925
Average train loss: 0.03301818041260141
Average train loss: 0.018422133449737085



##### **Predictions over the test set**

In [ ]:
label_indices_sent_cc = [] # These are the predictions
tokens_sent_cc = []
for cc in range(len(test_inputs_by_cc)):
  test_in = torch.tensor(test_inputs_by_cc[cc]).cuda()
  test_m = torch.tensor(test_masks_by_cc[cc]).cuda()
  with torch.no_grad(): # we don't want to optimize
    output = model(test_in, token_type_ids = None, attention_mask = test_m)
    #logits = output[1].detach().cpu().numpy()
  label_indices_sent_cc.append(np.argmax(output[0].to('cpu').numpy(), axis=2))
  #label_indices_sent_cc.append([list(p) for p in np.argmax(logits, axis=2)])

# convert input sequence into tokens
  tokens_sent_cc.append([tokenizer.convert_ids_to_tokens(ids.to('cpu').numpy()) for ids in test_in])

In [ ]:
new_tokens_cc, new_labels_cc, new_start_pos_cc = [], [], []
new_tokens_all, new_labels_all, new_start_pos_all = [], [], []

#dev_tags_by_cc2 contains the true labels of each token

for cc in range(len(test_inputs_by_cc)):
  new_tokens, new_labels, new_start_pos = [], [], []
  new_tokens, new_labels, new_start_pos = tokens_to_words_test(tokens_sent_cc[cc], 
                                        label_indices_sent_cc[cc], start_test_by_cc[cc], tag_values)
  new_tokens_cc.append(new_tokens)
  new_labels_cc.append(new_labels)
  new_start_pos_cc.append(new_start_pos)

  new_tokens_all.extend(new_tokens)
  new_labels_all.extend(new_labels)
  new_start_pos_all.extend(new_start_pos)

In [ ]:
# Example:
for token, label, new_start in zip(new_tokens_cc[6][5], new_labels_cc[6][5], new_start_pos_cc[6][5]):
    print("{}\t{}\t{}".format(label, token,new_start))

O	A	593
O	pesar	595
O	de	601
O	tener	604
O	unos	610
O	índices	615
O	elevados	623
O	de	632
O	creatina	635
O	-	635
O	colina	635
O	/	650
O	citrato	651
O	sugestivos	659
O	de	670
S-MOR	neoplasia	673
O	,	682
O	los	684
O	voxels	688
O	36	695
O	y	698
O	45	700
O	son	703
O	poco	707
O	valorables	712
O	puesto	723
O	que	730
O	corresponden	734
O	a	747
O	la	749
O	zona	752
O	de	757
O	uretra	760
O	prostática	767
O	.	777


In [ ]:
!ls 'drive/My Drive/Ejemplos NER - TFM/results_bert/'

ann  predictions


In [ ]:
import pickle as pkl

with open(path+'results_bert/predictions/new_tokens_cc', 'wb') as file: 
  pkl.dump(new_tokens_cc, file)
file.close()

with open(path+'results_bert/predictions/new_labels_cc', 'wb') as file: 
  pkl.dump(new_labels_cc, file)
file.close()

with open(path+'results_bert/predictions/new_start_pos_cc', 'wb') as file: 
  pkl.dump(new_start_pos_cc, file)
file.close()